In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import glob
import json
import gc

pd.options.display.max_rows = 10

## Implementation of metrics

In [2]:
from sklearn.metrics import roc_curve
from scipy.interpolate import interp1d

def rej_fixed_eff(truth, score, weight, efficiencies):
    fpr, tpr, thr = roc_curve(truth, score, sample_weight=weight)
    nonzero = (fpr != 0)
    eff = tpr[nonzero]
    rej = 1.0 / fpr[nonzero]
    
    interpol = interp1d(eff, rej, copy=False)
    return interpol(efficiencies)                    

In [3]:
# KS test is on hold
from scipy.stats import ks_2samp

def steps(x, y, inp):
    x = np.r_[-np.inf, x]
    y = np.r_[0.0, y]
    idx = np.searchsorted(x, inp, "right") - 1
    return y[idx]

def ks_2samp_weights(data1, data2, weight1, weight2):
    # Sort
    sarg1 = np.argsort(data1)
    sarg2 = np.argsort(data2)
    
    data1 = data1[sarg1]
    weight1 = weight1[sarg1]
    
    data2 = data2[sarg2]
    weight2 = weight2[sarg2]
    
    # Calculate cdf
    cdf1 = np.cumsum(weight1)
    cdf1 /= cdf1[-1]
    cdf2 = np.cumsum(weight2)
    cdf2 /= cdf2[-1]
    
    # Calculate all the ecdf values   
    x_all = np.concatenate([data1, data2])
    
    y1 = steps(data1, cdf1, x_all)
    y2 = steps(data2, cdf2, x_all)
    
    d = np.max(np.absolute(y1 - y2))
    
    s1 = np.sum(weight1)
    s2 = np.sum(weight2)
    
    en = np.sqrt(s1) * np.sqrt(s2) / np.sqrt(s1 + s2)
    
    try:
        prob = distributions.kstwobign.sf((en + 0.12 + 0.11 / en) * d)
    except:
        prob = 1.0
    
    return d, prob    

In [4]:
tid_1pvars = ["TauJets.centFrac", "TauJets.etOverPtLeadTrk",
              "TauJets.innerTrkAvgDist", "TauJets.absipSigLeadTrk",
              "TauJets.SumPtTrkFrac", "TauJets.ChPiEMEOverCaloEME",
              "TauJets.EMPOverTrkSysP", "TauJets.ptRatioEflowApprox",
              "TauJets.mEflowApprox"]

## Define analysis

In [5]:
train_sample = pd.read_hdf("samples/train.h5", columns=tid_1pvars + ["weight", "is_sig"])

train_weights = train_sample["weight"].get_values()
train_is_sig = train_sample["is_sig"].get_values()
dtrain = xgb.DMatrix(train_sample[tid_1pvars], label=train_sample["is_sig"], weight=train_sample["weight"])

%xdel train_sample
gc.collect()

test_sample = pd.read_hdf("samples/test.h5", columns=tid_1pvars + ["weight", "is_sig"])

test_weights = test_sample["weight"].get_values()
test_is_sig = test_sample["is_sig"].get_values()
dtest = xgb.DMatrix(test_sample[tid_1pvars], label=test_sample["is_sig"], weight=test_sample["weight"])

%xdel test_sample
gc.collect()

170

In [6]:
def do_analysis(filename):
    # Collection of metrics
    ret = {}
    
    # Load stuff
    with open(filename) as f:
        model_desc = json.load(f)
    
    bst = xgb.Booster(model_file="models/{}.model".format(model_desc["identifier"]))
    ret["best_iteration"] = int(bst.attributes()["best_iteration"])
    
    train_scores = bst.predict(dtrain, ntree_limit=ret["best_iteration"])
    test_scores = bst.predict(dtest, ntree_limit=ret["best_iteration"])
    
    rej30, rej50, rej70 = rej_fixed_eff(test_is_sig, test_scores, test_weights, [0.3, 0.5, 0.7])
    
    ret["id"] = model_desc["identifier"]
    ret["rej30"] = rej30
    ret["rej50"] = rej50
    ret["rej70"] = rej70

    
    ret.update(model_desc["config"])
    
    return ret

## Select processed files

In [7]:
processed = []
for model_desc in glob.glob("models/*.json"):
    with open(model_desc) as f:
        desc = json.load(f)
    if desc["processed"]:
        processed.append(model_desc)        

## Loop over analysis

In [ ]:
rets = []

In [ ]:
for model_desc in processed:
    ret = do_analysis(model_desc)
    print(ret)
    rets.append(ret)

In [ ]:
x = pd.DataFrame(rets)

In [ ]:
x.sort_values("rej50", ascending=False)